In [116]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 2000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', None)

In [2]:
df = pd.read_csv('/kaggle/input/trexquant-2025-summer-intern-challenge-iitkgp/TQ_Train_Kaggle.csv')
df

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,target
0,1,1.65,-3.48,NaN,0.12,0.22,0.20,NaN,-0.29,-0.36,-1.88,-2.82,NaN,NaN,NaN,0.09,-0.72,0.12,1.11,3.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,-0.41,0.44,0.00,NaN,-1.66,-0.63,NaN,NaN,0.77,NaN,NaN,NaN,0.79,NaN,NaN,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,0.39,2.82,3.52,NaN,0.72,0.62,-1.01,-0.97,0.02,NaN,0.02,NaN,1.47,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.26,-2.20,0.66,-2.20,NaN,0.03,0.01,0.30,-0.14,-1.95,NaN,0.99,-0.08
1,2,2.55,-0.93,NaN,0.17,0.27,0.08,NaN,-0.40,-0.54,-1.87,1.22,NaN,NaN,0.08,0.85,-0.76,-0.02,2.01,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,NaN,NaN,-2.11,NaN,NaN,0.66,0.96,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,-1.22,-0.22,-3.60,NaN,0.91,-0.97,-0.77,-0.97,0.02,NaN,0.01,-0.09,-3.61,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.24,-0.80,0.57,-0.81,NaN,0.16,0.00,0.19,1.18,0.12,NaN,-0.22,0.06
2,3,-3.73,0.99,NaN,0.15,0.16,0.40,NaN,-0.66,1.51,-1.66,-0.90,NaN,NaN,1.00,0.59,-3.18,-0.71,-2.74,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,NaN,NaN,0.99,NaN,NaN,0.67,0.46,NaN,NaN,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,0.44,-0.55,3.35,NaN,1.52,0.68,-0.85,0.37,0.02,NaN,0.11,0.22,0.45,NaN,0.00,-1.14,NaN,-4.00,NaN,NaN,NaN,0.76,NaN,0.22,0.15,0.52,0.21,NaN,0.04,0.00,0.18,-0.73,-0.37,NaN,0.36,0.00
3,4,2.12,0.04,NaN,0.12,0.76,-0.02,NaN,-0.27,-1.61,-1.77,-1.16,NaN,NaN,-0.22,0.43,-3.99,0.73,0.27,3.09,0.93,-0.98,-0.04,NaN,NaN,NaN,NaN,NaN,-0.09,NaN,0.09,0.35,0.44,0.00,1.53,1.90,0.24,NaN,NaN,0.10,NaN,NaN,0.81,0.65,NaN,NaN,NaN,NaN,NaN,NaN,-0.49,NaN,NaN,3.45,0.87,3.03,NaN,1.41,0.02,2.46,-0.97,0.02,-1.61,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,NaN,-4.00,NaN,NaN,NaN,0.55,NaN,0.19,-0.37,0.49,-0.40,NaN,0.08,0.00,0.33,-0.40,-0.19,NaN,0.01,-0.03
4,5,-0.85,-0.02,NaN,0.22,0.12,-0.27,NaN,1.56,-0.36,-2.62,-1.59,NaN,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,1.71,0.42,-0.83,NaN,NaN,NaN,NaN,NaN,0.01,NaN,0.09,0.21,0.43,0.00,1.49,1.88,NaN,NaN,NaN,0.70,NaN,NaN,0.87,2.47,NaN,NaN,NaN,NaN,NaN,NaN,-0.65,NaN,NaN,1.20,2.58,3.76,NaN,0.93,0.90,-1.49,0.40,0.02,NaN,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,NaN,0.06,NaN,NaN,NaN,0.34,NaN,0.06,1.64,0.23,1.50,NaN,0.05,0.00,0.54,0.51,-0.08,NaN,-0.01,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625,1.14,1.10,1.11,0.07,0.00,0.50,-3.97,-0.10,-0.14,-3.05,-3.51,-2.07,NaN,NaN,NaN,-2.19,-0.19,1.33,-1.53,NaN,NaN,NaN,1.37,1.12,-1.67,0.21,-0.47,-0.34,NaN,0.05,0.43,0.86,1.00,0.34,3.12,NaN,-1.41,NaN,-1.44,-1.19,NaN,NaN,0.34,0.56,0.25,NaN,NaN,NaN,0.18,-1.02,0.31,2.77,-4.00,-2.42,3.92,NaN,-0.87,0.24,-0.04,-0.97,0.24,NaN,NaN,NaN,-4.00,NaN,0.01,0.37,NaN,0.00,NaN,0.50,-1.33,0.42,0.28,0.08,0.13,0.19,-0.80,NaN,0.14,0.02,0.46,NaN,0.10,3.48,-0.28,0.00
105438,173659,-1.37,0.29,NaN,0.06,0.00,0.16,NaN,0.34,0.24,-2.28,-1.51,-0.46,NaN,NaN,NaN,NaN,0.09,0.69,0.04,NaN,NaN,NaN,-2.43,1.14,-4.00,2.44,-0.18,NaN,NaN,0.25,0.24,NaN,0.00,0.00,2.78,-0.04,NaN,NaN,NaN,0.52,NaN,NaN,0.55,-1.46,-0.62,NaN,NaN,NaN,0.34,1.89,0.39,-0.12,1.31,-0.05,NaN,NaN,NaN,-0.78,NaN,-0.78,-0.24,NaN,NaN,NaN,2.78,NaN,-0.03,0.50,NaN,0.00,NaN,0.12,1.80,0.22,0.06,NaN,NaN,0.19,-1.41,NaN,NaN,NaN,NaN,NaN,0.92,1.09,0.98,-0.01
105439,173670,0.05,0.34,NaN,0.08,0.22,-0.16,NaN,0.18,0.18,-1.16,-2.15,-1.16,NaN,NaN,NaN,-2.17,-0.42,1.29,-1.02,NaN,NaN,NaN,-3.83,-0.30,-2.54,0.84,-0.37,-0.53,NaN,0.19,0.28,-1.21,0.00,0.34,3.12,-0.01,-2.00,NaN,NaN,0.47,NaN,NaN,0.42,-1.32,-0.38,-0.50

In [3]:
feats = list(df.drop(columns=['row_id', 'target']).columns)
feats

['f1',
 'f2',
 'f3',
 'f4',
 'f5',
 'f6',
 'f7',
 'f8',
 'f9',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f19',
 'f20',
 'f21',
 'f22',
 'f23',
 'f24',
 'f25',
 'f26',
 'f27',
 'f28',
 'f29',
 'f30',
 'f31',
 'f32',
 'f33',
 'f34',
 'f35',
 'f36',
 'f37',
 'f38',
 'f39',
 'f40',
 'f41',
 'f42',
 'f43',
 'f44',
 'f45',
 'f46',
 'f47',
 'f48',
 'f49',
 'f50',
 'f51',
 'f52',
 'f53',
 'f54',
 'f55',
 'f56',
 'f57',
 'f58',
 'f59',
 'f60',
 'f61',
 'f62',
 'f63',
 'f64',
 'f65',
 'f66',
 'f67',
 'f68',
 'f69',
 'f70',
 'f71',
 'f72',
 'f73',
 'f74',
 'f75',
 'f76',
 'f77',
 'f78',
 'f79',
 'f80',
 'f81',
 'f82',
 'f83',
 'f84',
 'f85',
 'f86',
 'f87']

In [4]:
corr_dict = {}

for i in range(len(feats) - 1):
    for j in range(i+1, len(feats)):
        add_feat = df[feats[i]] + df[feats[j]]
        sub_feat = df[feats[i]] - df[feats[j]]
        mul_feat = df[feats[i]] * df[feats[j]]
        div_feat = df[feats[i]] / df[feats[j]].replace(0, 1e-9)
        
        add_corr = add_feat.corr(df['target'])
        sub_corr = sub_feat.corr(df['target'])
        mul_corr = mul_feat.corr(df['target'])
        div_corr = div_feat.corr(df['target'])
        
        corr_dict[f'{feats[i]} + {feats[j]}'] = add_corr
        corr_dict[f'{feats[i]} - {feats[j]}'] = sub_corr
        corr_dict[f'{feats[i]} * {feats[j]}'] = mul_corr
        corr_dict[f'{feats[i]} / {feats[j]}'] = div_corr

In [5]:
len(corr_dict)

14964

In [6]:
low, up = 0.07, 0.3
imp_feats = [key for key, value in corr_dict.items() if low <= value and value <= up]
imp_feats

['f20 + f41', 'f20 - f41', 'f41 / f47']

In [7]:
for feat in feats:
    log_corr = pd.Series(np.log(df[feat])).corr(df['target'])
    exp_corr = pd.Series(np.exp(df[feat])).corr(df['target'])
    sq_corr = pd.Series((df[feat])**2).corr(df['target'])
    sqrt_corr = pd.Series(np.sqrt(df[feat])).corr(df['target'])
    
    corr_dict[f'log({feat})'] = log_corr
    corr_dict[f'exp({feat})'] = exp_corr
    corr_dict[f'sq({feat})'] = sq_corr
    corr_dict[f'sqrt({feat})'] = sqrt_corr

/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:2742: RuntimeWarning: invalid value encountered in subtract
  X -= avg[:, None]


In [8]:
low, up = 0.07, 0.3
imp_feats = [key for key, value in corr_dict.items() if low <= value]
imp_feats

['f20 + f41', 'f20 - f41', 'f41 / f47']

In [9]:
len(corr_dict)

15312

In [10]:
df_test = pd.read_csv('/kaggle/input/trexquant-2025-summer-intern-challenge-iitkgp/TQ_Test_Kaggle.csv')
df_test

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87
0,44,-0.87,0.39,1.80,0.18,0.03,0.11,0.22,0.03,-1.02,-2.49,-1.87,-0.44,-0.07,0.85,1.18,-1.04,0.24,1.12,1.10,NaN,NaN,-0.40,-0.70,-0.02,1.01,-0.63,-0.44,0.08,0.01,0.11,0.00,0.83,2.00,1.61,2.78,-0.03,-1.33,-0.83,-0.05,0.18,-0.67,-0.89,-1.95,0.06,0.02,0.66,0.47,-2.36,0.06,1.74,0.26,-0.22,0.34,-0.45,3.52,0.00,1.07,0.97,-1.90,0.27,0.62,NaN,0.00,-0.03,-1.81,-0.05,0.08,-0.40,0.26,-0.13,0.18,0.22,-2.27,0.47,0.39,0.13,-2.02,0.25,-2.01,-0.70,0.11,0.00,0.12,1.17,-0.96,3.70,0.05
1,45,-1.07,-0.00,2.83,0.10,0.07,0.26,NaN,NaN,0.71,-1.41,-1.72,0.54,0.49,0.77,-0.12,-0.49,0.77,1.12,-0.11,NaN,NaN,-1.01,0.27,0.66,-1.44,-0.95,-0.40,-0.12,-0.15,0.10,0.01,0.91,0.00,1.55,3.30,-0.03,-2.00,0.63,-0.03,-0.76,-1.69,-0.89,-0.89,-0.47,0.06,-2.31,-0.98,-2.44,0.06,0.96,0.29,0.01,1.07,0.89,-1.82,NaN,0.34,0.87,1.41,1.00,0.33,NaN,0.01,0.00,0.91,0.14,0.00,-0.54,0.18,-0.06,0.50,0.41,-2.46,0.48,0.64,0.12,-1.65,0.28,-1.65,1.38,0.12,0.00,0.13,0.87,-0.70,3.72,0.20
2,46,-0.61,0.99,-2.31,0.16,0.40,0.05,-3.01,-1.50,0.36,-2.20,0.16,0.39,0.42,0.25,-0.64,-0.58,-0.54,1.21,1.96,NaN,1.37,1.72,0.62,0.42,-0.97,0.37,-0.49,-0.12,-0.27,0.09,0.02,0.86,1.00,1.65,3.11,-0.05,-2.93,-1.71,-0.85,-1.51,0.13,-0.89,-1.19,-0.14,-0.07,-3.61,-3.59,-2.17,0.13,2.11,0.13,0.86,0.00,1.92,-0.28,NaN,0.85,0.33,-1.98,-0.87,0.17,NaN,0.03,0.06,-2.62,0.12,0.06,-0.34,0.25,0.23,1.00,0.19,-1.71,0.52,0.36,0.15,1.20,0.31,1.11,-0.57,0.12,0.00,0.09,1.03,-0.53,3.79,0.11
3,47,2.63,-0.34,0.48,0.15,0.10,-0.02,NaN,1.71,0.77,-1.29,-0.95,0.80,0.44,1.28,0.25,-0.55,-0.05,1.48,1.34,NaN,0.41,1.06,1.63,0.22,-0.27,1.84,0.32,-0.20,-0.06,0.09,0.02,0.82,2.00,1.74,0.52,-0.05,-2.64,NaN,0.64,-0.23,-0.01,-0.89,1.37,0.05,0.06,1.78,1.23,-1.73,0.24,0.20,0.86,0.57,2.80,3.11,-3.19,NaN,1.97,1.00,0.77,1.00,0.62,NaN,0.59,0.08,2.35,0.09,-0.05,-1.03,0.39,0.00,0.86,0.29,-2.36,0.31,0.43,0.12,-0.47,0.23,-0.48,2.23,0.11,0.00,0.10,1.97,-0.51,3.64,0.08
4,48,0.66,-0.85,-0.98,0.18,1.15,0.48,NaN,0.50,-0.62,-0.58,-2.57,-1.06,NaN,0.30,0.63,-0.97,0.00,1.05,-0.53,NaN,-0.34,-0.43,-0.75,-0.17,0.64,0.80,0.24,-0.11,0.43,0.08,0.03,0.82,0.00,0.95,1.94,-0.05,-2.40,NaN,-0.36,-0.66,0.80,0.13,-1.47,0.91,0.12,-0.03,-0.03,-1.24,0.39,1.63,0.24,0.15,-2.50,-1.39,3.03,NaN,1.50,0.62,1.33,-0.84,-0.21,NaN,NaN,0.02,0.02,0.07,0.05,-0.25,0.24,-0.41,2.52,0.33,-2.27,0.35,0.32,0.13,-1.31,0.26,-1.33,NaN,0.39,0.00,0.13,2.22,-0.68,3.72,0.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85816,191259,-0.37,-0.35,-0.40,0.09,0.65,0.47,NaN,0.00,-0.32,-2.56,-2.19,-0.55,NaN,NaN,-0.22,-3.58,-0.16,0.88,1.93,NaN,NaN,0.58,-0.45,0.62,-1.36,-0.13,-0.49,-0.16,0.27,0.32,-0.11,-1.10,1.00,0.04,2.88,-0.02,-1.33,-0.71,-3.46,0.53,1.29,-0.12,0.42,-0.09,-0.12,NaN,NaN,-1.87,0.09,2.94,0.30,0.61,1.11,-1.13,-3.27,NaN,1.10,0.52,2.14,-0.46,-0.02,NaN,0.00,NaN,1.13,NaN,0.11,0.22,NaN,-0.03,NaN,0.12,-0.79,0.26,0.49,0.07,0.71,0.15,0.45,NaN,0.02,0.00,0.78,0.49,2.27,3.49,0.55
85817,191260,0.75,1.22,NaN,0.02,0.14,-0.21,NaN,1.58,-0.04,-2.82,-1.76,-1.96,NaN,NaN,NaN,-1.22,-0.82,0.44,4.00,NaN,NaN,NaN,0.20,0.14,-0.80,1.25,0.44,-0.06,0.31,0.11,0.10,-1.00,1.00,0.05,1.38,-0.02,-2.00,-0.70,1.71,NaN,NaN,-0.56,-0.57,-0.50,-0.03,NaN,NaN,-2.20,0.11,1.97,0.95,-0.61,-0.70,1.64,-0.36,NaN,-1.83,-0.97,-0.28,0.94,0.14,2.04,0.03,NaN,4.00,NaN,-0.19,0.56,NaN,-0.07,NaN,0.16,-1.58,0.30,0.37,0.06,-0.15,0.14,-0.10,-0.89,0.13,0.00,0.48,NaN,2.11,2.94,-0.10
85818,191261,-0.04,0.39,-0.86,0.05,0.07,-0.03,N

# SMA & EMA

In [11]:
# Calculate SMA and EMA for the target column
window = 10
df['SMA'] = df['target'].rolling(window=window, min_periods=1).mean()
df['EMA'] = df['target'].ewm(span=window, adjust=False, min_periods=1).mean()

In [12]:
df

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,target,SMA,EMA
0,1,1.65,-3.48,NaN,0.12,0.22,0.20,NaN,-0.29,-0.36,-1.88,-2.82,NaN,NaN,NaN,0.09,-0.72,0.12,1.11,3.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,-0.41,0.44,0.00,NaN,-1.66,-0.63,NaN,NaN,0.77,NaN,NaN,NaN,0.79,NaN,NaN,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,0.39,2.82,3.52,NaN,0.72,0.62,-1.01,-0.97,0.02,NaN,0.02,NaN,1.47,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.26,-2.20,0.66,-2.20,NaN,0.03,0.01,0.30,-0.14,-1.95,NaN,0.99,-0.08,-0.08,-0.08
1,2,2.55,-0.93,NaN,0.17,0.27,0.08,NaN,-0.40,-0.54,-1.87,1.22,NaN,NaN,0.08,0.85,-0.76,-0.02,2.01,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,NaN,NaN,-2.11,NaN,NaN,0.66,0.96,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,-1.22,-0.22,-3.60,NaN,0.91,-0.97,-0.77,-0.97,0.02,NaN,0.01,-0.09,-3.61,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.24,-0.80,0.57,-0.81,NaN,0.16,0.00,0.19,1.18,0.12,NaN,-0.22,0.06,-0.01,-0.05
2,3,-3.73,0.99,NaN,0.15,0.16,0.40,NaN,-0.66,1.51,-1.66,-0.90,NaN,NaN,1.00,0.59,-3.18,-0.71,-2.74,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,NaN,NaN,0.99,NaN,NaN,0.67,0.46,NaN,NaN,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,0.44,-0.55,3.35,NaN,1.52,0.68,-0.85,0.37,0.02,NaN,0.11,0.22,0.45,NaN,0.00,-1.14,NaN,-4.00,NaN,NaN,NaN,0.76,NaN,0.22,0.15,0.52,0.21,NaN,0.04,0.00,0.18,-0.73,-0.37,NaN,0.36,0.00,-0.01,-0.04
3,4,2.12,0.04,NaN,0.12,0.76,-0.02,NaN,-0.27,-1.61,-1.77,-1.16,NaN,NaN,-0.22,0.43,-3.99,0.73,0.27,3.09,0.93,-0.98,-0.04,NaN,NaN,NaN,NaN,NaN,-0.09,NaN,0.09,0.35,0.44,0.00,1.53,1.90,0.24,NaN,NaN,0.10,NaN,NaN,0.81,0.65,NaN,NaN,NaN,NaN,NaN,NaN,-0.49,NaN,NaN,3.45,0.87,3.03,NaN,1.41,0.02,2.46,-0.97,0.02,-1.61,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,NaN,-4.00,NaN,NaN,NaN,0.55,NaN,0.19,-0.37,0.49,-0.40,NaN,0.08,0.00,0.33,-0.40,-0.19,NaN,0.01,-0.03,-0.01,-0.04
4,5,-0.85,-0.02,NaN,0.22,0.12,-0.27,NaN,1.56,-0.36,-2.62,-1.59,NaN,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,1.71,0.42,-0.83,NaN,NaN,NaN,NaN,NaN,0.01,NaN,0.09,0.21,0.43,0.00,1.49,1.88,NaN,NaN,NaN,0.70,NaN,NaN,0.87,2.47,NaN,NaN,NaN,NaN,NaN,NaN,-0.65,NaN,NaN,1.20,2.58,3.76,NaN,0.93,0.90,-1.49,0.40,0.02,NaN,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,NaN,0.06,NaN,NaN,NaN,0.34,NaN,0.06,1.64,0.23,1.50,NaN,0.05,0.00,0.54,0.51,-0.08,NaN,-0.01,0.06,0.00,-0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625,1.14,1.10,1.11,0.07,0.00,0.50,-3.97,-0.10,-0.14,-3.05,-3.51,-2.07,NaN,NaN,NaN,-2.19,-0.19,1.33,-1.53,NaN,NaN,NaN,1.37,1.12,-1.67,0.21,-0.47,-0.34,NaN,0.05,0.43,0.86,1.00,0.34,3.12,NaN,-1.41,NaN,-1.44,-1.19,NaN,NaN,0.34,0.56,0.25,NaN,NaN,NaN,0.18,-1.02,0.31,2.77,-4.00,-2.42,3.92,NaN,-0.87,0.24,-0.04,-0.97,0.24,NaN,NaN,NaN,-4.00,NaN,0.01,0.37,NaN,0.00,NaN,0.50,-1.33,0.42,0.28,0.08,0.13,0.19,-0.80,NaN,0.14,0.02,0.46,NaN,0.10,3.48,-0.28,0.00,-0.01,0.00
105438,173659,-1.37,0.29,NaN,0.06,0.00,0.16,NaN,0.34,0.24,-2.28,-1.51,-0.46,NaN,NaN,NaN,NaN,0.09,0.69,0.04,NaN,NaN,NaN,-2.43,1.14,-4.00,2.44,-0.18,NaN,NaN,0.25,0.24,NaN,0.00,0.00,2.78,-0.04,NaN,NaN,NaN,0.52,NaN,NaN,0.55,-1.46,-0.62,NaN,NaN,NaN,0.34,1.89,0.39,-0.12,1.31,-0.05,NaN,NaN,NaN,-0.78,NaN,-0.78,-0.24,NaN,NaN,NaN,2.78,NaN,-0.03,0.50,NaN,0.00,NaN,0.12,1.80,0.22,0.06,NaN,NaN,0.19,-1.41,NaN,NaN,NaN,NaN,NaN,0.92,1.09,0.98,-0.01,-0.00,0.00
105439,173670,0.05,0.34,NaN,0.08,0.22,-0.16,NaN,0.18,0.18,-1.16,-2.15,-1.16,NaN,NaN,NaN,-2.17,-0.42,1.29,-1.02,NaN,NaN,NaN,-3.83,-0.30,-2.54,0.84,-0.37

In [13]:
df['EMA'].corr(df['target'])

0.5749230276317719

In [14]:
from sklearn.metrics import mean_squared_error
mean_squared_error(df['EMA'], df['target'])

0.0031591607420118507

In [15]:
df_train_test = pd.concat([df, df_test]).sort_values(by='row_id')
df_train_test.head(100)

,row_id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,target,SMA,EMA
0,1,1.65,-3.48,NaN,0.12,0.22,0.20,NaN,-0.29,-0.36,-1.88,-2.82,NaN,NaN,NaN,0.09,-0.72,0.12,1.11,3.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.41,-0.41,0.44,0.00,NaN,-1.66,-0.63,NaN,NaN,0.77,NaN,NaN,NaN,0.79,NaN,NaN,NaN,NaN,NaN,NaN,-0.25,NaN,NaN,0.39,2.82,3.52,NaN,0.72,0.62,-1.01,-0.97,0.02,NaN,0.02,NaN,1.47,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.26,-2.20,0.66,-2.20,NaN,0.03,0.01,0.30,-0.14,-1.95,NaN,0.99,-0.08,-0.08,-0.08
1,2,2.55,-0.93,NaN,0.17,0.27,0.08,NaN,-0.40,-0.54,-1.87,1.22,NaN,NaN,0.08,0.85,-0.76,-0.02,2.01,0.11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,NaN,NaN,-2.11,NaN,NaN,0.66,0.96,NaN,NaN,NaN,NaN,NaN,NaN,2.19,NaN,NaN,-1.22,-0.22,-3.60,NaN,0.91,-0.97,-0.77,-0.97,0.02,NaN,0.01,-0.09,-3.61,NaN,0.00,-3.11,NaN,-4.00,NaN,NaN,NaN,0.50,NaN,0.24,-0.80,0.57,-0.81,NaN,0.16,0.00,0.19,1.18,0.12,NaN,-0.22,0.06,-0.01,-0.05
2,3,-3.73,0.99,NaN,0.15,0.16,0.40,NaN,-0.66,1.51,-1.66,-0.90,NaN,NaN,1.00,0.59,-3.18,-0.71,-2.74,3.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,NaN,NaN,0.99,NaN,NaN,0.67,0.46,NaN,NaN,NaN,NaN,NaN,NaN,-0.13,NaN,NaN,0.44,-0.55,3.35,NaN,1.52,0.68,-0.85,0.37,0.02,NaN,0.11,0.22,0.45,NaN,0.00,-1.14,NaN,-4.00,NaN,NaN,NaN,0.76,NaN,0.22,0.15,0.52,0.21,NaN,0.04,0.00,0.18,-0.73,-0.37,NaN,0.36,0.00,-0.01,-0.04
3,4,2.12,0.04,NaN,0.12,0.76,-0.02,NaN,-0.27,-1.61,-1.77,-1.16,NaN,NaN,-0.22,0.43,-3.99,0.73,0.27,3.09,0.93,-0.98,-0.04,NaN,NaN,NaN,NaN,NaN,-0.09,NaN,0.09,0.35,0.44,0.00,1.53,1.90,0.24,NaN,NaN,0.10,NaN,NaN,0.81,0.65,NaN,NaN,NaN,NaN,NaN,NaN,-0.49,NaN,NaN,3.45,0.87,3.03,NaN,1.41,0.02,2.46,-0.97,0.02,-1.61,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,NaN,-4.00,NaN,NaN,NaN,0.55,NaN,0.19,-0.37,0.49,-0.40,NaN,0.08,0.00,0.33,-0.40,-0.19,NaN,0.01,-0.03,-0.01,-0.04
4,5,-0.85,-0.02,NaN,0.22,0.12,-0.27,NaN,1.56,-0.36,-2.62,-1.59,NaN,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,1.71,0.42,-0.83,NaN,NaN,NaN,NaN,NaN,0.01,NaN,0.09,0.21,0.43,0.00,1.49,1.88,NaN,NaN,NaN,0.70,NaN,NaN,0.87,2.47,NaN,NaN,NaN,NaN,NaN,NaN,-0.65,NaN,NaN,1.20,2.58,3.76,NaN,0.93,0.90,-1.49,0.40,0.02,NaN,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,NaN,0.06,NaN,NaN,NaN,0.34,NaN,0.06,1.64,0.23,1.50,NaN,0.05,0.00,0.54,0.51,-0.08,NaN,-0.01,0.06,0.00,-0.02
5,6,0.62,0.92,NaN,0.07,0.22,-0.44,NaN,-1.14,-1.13,-1.01,-1.79,NaN,0.84,0.57,0.54,-1.25,-0.22,0.77,1.06,-0.08,-0.40,1.46,NaN,NaN,NaN,NaN,NaN,-0.08,NaN,0.08,0.02,0.51,0.00,1.78,3.06,-0.08,NaN,NaN,0.48,NaN,NaN,0.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.47,NaN,NaN,1.46,0.74,0.04,NaN,-1.48,0.52,1.09,0.37,0.02,NaN,NaN,0.06,-0.19,0.00,0.05,-1.09,NaN,0.09,NaN,NaN,NaN,0.31,NaN,0.11,-1.00,0.30,-1.03,NaN,0.03,0.00,0.52,1.60,0.24,NaN,-0.21,-0.09,-0.01,-0.04
6,7,-0.76,1.02,-0.79,0.18,0.36,-0.40,NaN,-0.20,1.15,-2.63,-2.76,NaN,1.03,0.38,0.25,-1.09,-0.42,-0.17,-1.09,-0.14,-0.25,0.34,NaN,NaN,NaN,NaN,NaN,-0.22,NaN,0.09,0.01,0.47,0.00,1.82,3.25,-0.16,NaN,NaN,0.33,NaN,NaN,0.62,-1.30,NaN,NaN,NaN,NaN,NaN,NaN,-0.24,NaN,NaN,-3.33,-3.29,-2.30,NaN,1.11,0.30,0.36,-0.24,0.02,2.13,0.12,-0.21,-4.00,-0.10,0.18,-1.14,NaN,0.00,NaN,NaN,NaN,0.22,NaN,0.17,1.71,0.37,1.72,NaN,0.11,0.00,0.61,2.07,0.52,NaN,-0.13,0.09,0.00,-0.01
7,8,1.93,0.41,0.33,0.13,0.31,0.32,NaN,0.94,-1.11,-2.42,-1.98,NaN,0.58,-0.44,0.47,-0.70,0.03,-0.38,3.41,NaN,0.07,0.46,-1.96,NaN,-1.76,1.47,-0.04,-0.37,-0.17,0.09,-0.00,0.47,0.00,1.39,2.43,-0.03,NaN,NaN,0.28,NaN,NaN,0.74,-0.86,NaN,NaN,NaN,NaN,NaN,NaN,0.54,NaN,0.00,3.51,-0.69,2.38,NaN,-1.64,-0.43,-1.25,-0.87,0.02,NaN,0.02,0.20,3.01,0.02,0.05,-3.20,NaN,0.11,-1.63,0.04,NaN,0.56,NaN,0.12,0.36,0.28,0.38,NaN,0.17,0.00,0.51,-0.36,0.88,NaN,-0.13,-0.02,-0.00,-0.01
8,9,0.12,-0.69,-0.41,0.12,0.32,0.16,NaN,1.68,0.31,-2.56,0.42,NaN,0.15,-0.52,-0.2

In [16]:
all_target = df_train_test['target']

In [37]:
X = np.array(df['target'])

range_start = 1
range_end = len(X)
num_samples = int(len(X)*0.2)

val_idx = np.array(sorted(np.random.choice(range(range_start, range_end + 1), num_samples, replace=False)))
val_idx

array([     9,     14,     29, ..., 105433, 105438, 105441])

In [38]:
X_val = X[val_idx]

X[val_idx] = np.nan

In [39]:
len(X_val)

21088

In [40]:
np.isnan(X[2])

False

In [41]:
def get_EMA(array, span):
    alpha = 2 / (span + 1)
    ema = array[0]  

    for i in range(1, len(array)):
        ema = alpha * array[i] + (1 - alpha) * ema
    
    return ema

In [42]:
span = 10

for i in range(len(X)):
    if np.isnan(X[i]):
        past = X[:i]
        X[i] = get_EMA(past, span)

In [43]:
X_pred = X[val_idx]
mean_squared_error(X_pred, X_val)

0.004764633349483802

# LSTM

In [45]:
df_knn = pd.read_csv('/kaggle/input/df-new-drop-knn-impute/df_new_drop_knn_impute.csv').drop(columns=['Unnamed: 0'])
df_knn

,row_id,f1,f2,f3,f4,f5,f6,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f21,f23,f24,f25,f26,f27,f28,f30,f31,f32,f33,f34,f35,f36,f37,f39,f41,f42,f43,f44,f45,f48,f49,f50,f51,f52,f53,f54,f55,f57,f58,f59,f60,f61,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f74,f75,f76,f77,f78,f79,f81,f82,f83,f84,f85,f86,f87,target
0,1.00,1.65,-3.48,-0.23,0.12,0.22,0.20,-0.29,-0.36,-1.88,-2.82,-1.00,0.30,0.29,0.09,-0.72,0.12,1.11,3.31,-0.24,-0.08,-0.22,-0.64,-0.08,-0.03,-0.17,0.41,-0.41,0.44,0.00,1.52,-1.66,-0.63,-1.36,0.77,-0.45,0.71,0.79,0.05,-0.06,-2.25,0.14,-0.25,0.16,-0.34,0.39,2.82,3.52,0.72,0.62,-1.01,-0.97,0.02,0.02,0.08,1.47,-0.08,0.00,-3.11,0.18,-4.00,-2.00,0.14,0.50,0.45,0.26,-2.20,0.66,-2.20,0.03,0.01,0.30,-0.14,-1.95,3.74,0.99,-0.08
1,2.00,2.55,-0.93,-0.34,0.17,0.27,0.08,-0.40,-0.54,-1.87,1.22,-1.12,0.67,0.08,0.85,-0.76,-0.02,2.01,0.11,-0.31,-0.08,-0.16,-0.64,-0.08,-0.03,-0.14,0.10,-0.10,0.49,0.00,1.24,3.02,-0.26,-1.50,-2.11,-0.45,0.66,0.96,0.05,-0.06,-1.66,0.14,2.19,0.17,-0.34,-1.22,-0.22,-3.60,0.91,-0.97,-0.77,-0.97,0.02,0.01,-0.09,-3.61,-0.11,0.00,-3.11,0.19,-4.00,-2.00,0.14,0.50,0.52,0.24,-0.80,0.57,-0.81,0.16,0.00,0.19,1.18,0.12,3.74,-0.22,0.06
2,3.00,-3.73,0.99,0.04,0.15,0.16,0.40,-0.66,1.51,-1.66,-0.90,-1.00,0.50,1.00,0.59,-3.18,-0.71,-2.74,3.03,-0.24,-0.49,-0.22,-0.65,0.42,-0.00,-0.17,0.09,-0.09,0.45,0.00,1.43,1.01,-0.26,-1.50,0.99,-0.45,0.67,0.46,0.05,-0.06,-2.25,0.14,-0.13,0.16,-0.20,0.44,-0.55,3.35,1.52,0.68,-0.85,0.37,0.02,0.11,0.22,0.45,-0.08,0.00,-1.14,0.18,-4.00,-1.88,0.08,0.76,0.45,0.22,0.15,0.52,0.21,0.04,0.00,0.18,-0.73,-0.37,3.74,0.36,0.00
3,4.00,2.12,0.04,0.31,0.12,0.76,-0.02,-0.27,-1.61,-1.77,-1.16,-1.00,0.30,-0.22,0.43,-3.99,0.73,0.27,3.09,-0.98,-0.20,-0.22,-0.63,0.49,-0.11,-0.09,0.09,0.35,0.44,0.00,1.53,1.90,0.24,-1.50,0.10,-0.45,0.81,0.65,-0.05,-0.05,-2.25,0.19,-0.49,0.16,-0.39,3.45,0.87,3.03,1.41,0.02,2.46,-0.97,0.02,1.00,-0.05,-0.91,-0.19,-0.28,-2.55,0.18,-4.00,-1.58,0.14,0.55,0.45,0.19,-0.37,0.49,-0.40,0.08,0.00,0.33,-0.40,-0.19,3.62,0.01,-0.03
4,5.00,-0.85,-0.02,0.04,0.22,0.12,-0.27,1.56,-0.36,-2.62,-1.59,-1.00,1.15,0.52,0.32,-2.02,0.88,-0.81,-0.07,0.42,-0.03,-0.22,-0.98,0.07,-0.19,0.01,0.09,0.21,0.43,0.00,1.49,1.88,-0.03,-1.50,0.70,-0.45,0.87,2.47,0.05,-0.06,-2.25,0.14,-0.65,0.16,-0.27,1.20,2.58,3.76,0.93,0.90,-1.49,0.40,0.02,0.03,-0.04,-0.94,-0.19,-0.05,-3.11,0.18,0.06,-1.63,0.12,0.34,0.45,0.06,1.64,0.23,1.50,0.05,0.00,0.54,0.51,-0.08,3.74,-0.01,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625.00,1.14,1.10,1.11,0.07,0.00,0.50,-0.10,-0.14,-3.05,-3.51,-2.07,-0.69,-0.21,0.08,-2.19,-0.19,1.33,-1.53,0.25,1.37,1.12,-1.67,0.21,-0.47,-0.34,0.05,0.43,0.86,1.00,0.34,3.12,0.02,-1.41,-1.44,-0.75,-0.73,0.34,0.56,0.25,-1.18,0.18,-1.02,0.31,2.77,-4.00,-2.42,3.92,-0.87,0.24,-0.04,-0.97,0.24,0.22,0.05,-4.00,0.00,0.01,0.37,0.23,0.00,-0.29,0.50,0.42,0.28,0.08,0.13,0.19,-0.80,0.14,0.02,0.46,1.00,0.10,3.48,-0.28,0.00
105438,173659.00,-1.37,0.29,-0.18,0.06,0.00,0.16,0.34,0.24,-2.28,-1.51,-0.46,-0.69,-0.21,0.08,-1.78,0.09,0.69,0.04,0.25,-2.43,1.14,-4.00,2.44,-0.18,-0.13,0.25,0.24,0.02,0.00,0.00,2.78,-0.04,-1.77,-0.83,-0.75,-0.73,0.55,-1.46,-0.62,-1.18,0.34,1.89,0.39,-0.12,1.31,-0.05,-0.65,-0.66,-0.78,0.40,-0.78,-0.24,0.22,0.05,2.78,0.00,-0.03,0.50,0.23,0.00,-0.18,0.12,0.22,0.06,0.09,-0.71,0.19,-1.41,0.18,0.01,0.25,1.00,0.92,1.09,0.98,-0.01
105439,173670.00,0.05,0.34,-0.18,0.08,0.22,-0.16,0.18,0.18,-1.16,-2.15,-1.16,-0.69,-0.21,0.08,-2.17,-0.42,1.29,-1.02,0.25,-3.83,-0.30,-2.54,0.84,-0.37,-0.53,0.19,0.28,-1.21,0.00,0.34,3.12,-0.01,-2.00,-1.20,-0.75,-0.73,0.42,-1.32,-0.38,-1.18,0.19,-0.86,0.32,0.82,0.54,-4.00,-2.87,-1.50,0.17,-0.04,-0.97,0.37,0.22,0.05,1.23,0.00,0.06,0.00,0.23,0.00,-0.29,0.37,0.23,0.19,0.09,-0.19,0.17,-0.28,0.09,0.01,0.03,1.00,-0.37,3.05,0.38

In [117]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

In [47]:
features = df_knn.drop(columns=['row_id', 'target'])
target = df_knn['target']

# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [48]:
# Function to create sequences
def create_sequences(features, target, sequence_length):
    X, y = [], []
    for i in range(len(features) - sequence_length):
        X.append(features[i:i+sequence_length])
        y.append(target[i+sequence_length])
    return np.array(X), np.array(y)

In [49]:
# Parameters
sequence_length = 10

# Create sequences
X, y = create_sequences(features_scaled, target.values, sequence_length)

# Split into training and test sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

In [55]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, features.shape[1]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 10, 50)         │        25,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 45,451 (177.54 KB)

 Trainable params: 45,451 (177.54 KB)

 Non-trainable params: 0 (0.00 B)

In [56]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 32s 11ms/step - loss: 0.0059 - val_loss: 0.0057
Epoch 2/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0039 - val_loss: 0.0057
Epoch 3/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0040 - val_loss: 0.0057
Epoch 4/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 28s 11ms/step - loss: 0.0039 - val_loss: 0.0057
Epoch 5/50
2263/2636 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0040

KeyboardInterrupt: 

In [57]:
lag_dict = {'f84': [],
 'f82': [1],
 'f24': [1, 2],
 'f16': [1, 2],
 'f8': [],
 'f64': [],
 'f59': [1],
 'f23': [],
 'f2': [],
 'f15': [],
 'f30': [1, 2],
 'f3': [1, 2],
 'f21': [],
 'f27': [],
 'f51': [],
 'f1': [],
 'f57': [],
 'f35': [1, 2],
 'f13': [1, 2],
 'f52': [],
 'f39': [],
 'f53': [],
 'f28': [1, 2],
 'f81': [1, 2],
 'f87': [1, 2],
 'f14': [],
 'f43': [],
 'f36': [1, 2],
 'f69': [],
 'f42': [1, 2],
 'f32': [1, 2],
 'f74': [],
 'f75': [],
 'f77': [],
 'f71': [1, 2],
 'f5': [1],
 'f66': [1],
 'f58': [],
 'f34': [1, 2],
 'f18': [1],
 'f17': [],
 'f63': [1, 2],
 'f83': [1]}

In [58]:
len(lag_dict)

43

In [59]:
df_lag_dict = {'row_id': np.array(df_knn['row_id'])}

for key, value in lag_dict.items():
    df_lag_dict[key] = np.array(df_knn[key])
    # create lag feature
    for lag in value:
        df_lag_dict[f'{key}_lag_{lag}'] = np.array(df_knn[key].shift(lag))
df_lag_dict['target'] = np.array(df_knn['target'])
df_lag = pd.DataFrame(df_lag_dict)
df_lag

,row_id,f84,f82,f82_lag_1,f24,f24_lag_1,f24_lag_2,f16,f16_lag_1,f16_lag_2,f8,f64,f59,f59_lag_1,f23,f2,f15,f30,f30_lag_1,f30_lag_2,f3,f3_lag_1,f3_lag_2,f21,f27,f51,f1,f57,f35,f35_lag_1,f35_lag_2,f13,f13_lag_1,f13_lag_2,f52,f39,f53,f28,f28_lag_1,f28_lag_2,f81,f81_lag_1,f81_lag_2,f87,f87_lag_1,f87_lag_2,f14,f43,f36,f36_lag_1,f36_lag_2,f69,f42,f42_lag_1,f42_lag_2,f32,f32_lag_1,f32_lag_2,f74,f75,f77,f71,f71_lag_1,f71_lag_2,f5,f5_lag_1,f66,f66_lag_1,f58,f34,f34_lag_1,f34_lag_2,f18,f18_lag_1,f17,f63,f63_lag_1,f63_lag_2,f83,f83_lag_1,target
0,1.00,-0.14,0.01,NaN,-0.22,NaN,NaN,-0.72,NaN,NaN,-0.29,0.08,-1.01,NaN,-0.08,-3.48,0.09,0.41,NaN,NaN,-0.23,NaN,NaN,-0.24,-0.03,0.16,1.65,0.72,-1.66,NaN,NaN,0.30,NaN,NaN,-0.34,0.77,0.39,-0.17,NaN,NaN,0.03,NaN,NaN,0.99,NaN,NaN,0.29,0.79,-0.63,NaN,NaN,0.18,0.71,NaN,NaN,0.44,NaN,NaN,0.50,0.45,-2.20,-2.00,NaN,NaN,0.22,NaN,-0.08,NaN,0.62,1.52,NaN,NaN,1.11,NaN,0.12,0.02,NaN,NaN,0.30,NaN,-0.08
1,2.00,1.18,0.00,0.01,-0.16,-0.22,NaN,-0.76,-0.72,NaN,-0.40,-0.09,-0.77,-1.01,-0.08,-0.93,0.85,0.10,0.41,NaN,-0.34,-0.23,NaN,-0.31,-0.03,0.17,2.55,0.91,3.02,-1.66,NaN,0.67,0.30,NaN,-0.34,-2.11,-1.22,-0.14,-0.17,NaN,0.16,0.03,NaN,-0.22,0.99,NaN,0.08,0.96,-0.26,-0.63,NaN,0.19,0.66,0.71,NaN,0.49,0.44,NaN,0.50,0.52,-0.80,-2.00,-2.00,NaN,0.27,0.22,-0.11,-0.08,-0.97,1.24,1.52,NaN,2.01,1.11,-0.02,0.01,0.02,NaN,0.19,0.30,0.06
2,3.00,-0.73,0.00,0.00,-0.22,-0.16,-0.22,-3.18,-0.76,-0.72,-0.66,0.22,-0.85,-0.77,-0.49,0.99,0.59,0.09,0.10,0.41,0.04,-0.34,-0.23,-0.24,-0.00,0.16,-3.73,1.52,1.01,3.02,-1.66,0.50,0.67,0.30,-0.20,0.99,0.44,-0.17,-0.14,-0.17,0.04,0.16,0.03,0.36,-0.22,0.99,1.00,0.46,-0.26,-0.26,-0.63,0.18,0.67,0.66,0.71,0.45,0.49,0.44,0.76,0.45,0.15,-1.88,-2.00,-2.00,0.16,0.27,-0.08,-0.11,0.68,1.43,1.24,1.52,-2.74,2.01,-0.71,0.11,0.01,0.02,0.18,0.19,0.00
3,4.00,-0.40,0.00,0.00,-0.22,-0.22,-0.16,-3.99,-3.18,-0.76,-0.27,-0.05,2.46,-0.85,-0.20,0.04,0.43,0.09,0.09,0.10,0.31,0.04,-0.34,-0.98,-0.11,0.16,2.12,1.41,1.90,1.01,3.02,0.30,0.50,0.67,-0.39,0.10,3.45,-0.09,-0.17,-0.14,0.08,0.04,0.16,0.01,0.36,-0.22,-0.22,0.65,0.24,-0.26,-0.26,0.18,0.81,0.67,0.66,0.44,0.45,0.49,0.55,0.45,-0.37,-1.58,-1.88,-2.00,0.76,0.16,-0.19,-0.08,0.02,1.53,1.43,1.24,0.27,-2.74,0.73,1.00,0.11,0.01,0.33,0.18,-0.03
4,5.00,0.51,0.00,0.00,-0.22,-0.22,-0.22,-2.02,-3.99,-3.18,1.56,-0.04,-1.49,2.46,-0.03,-0.02,0.32,0.09,0.09,0.09,0.04,0.31,0.04,0.42,-0.19,0.16,-0.85,0.93,1.88,1.90,1.01,1.15,0.30,0.50,-0.27,0.70,1.20,0.01,-0.09,-0.17,0.05,0.08,0.04,-0.01,0.01,0.36,0.52,2.47,-0.03,0.24,-0.26,0.18,0.87,0.81,0.67,0.43,0.44,0.45,0.34,0.45,1.64,-1.63,-1.58,-1.88,0.12,0.76,-0.19,-0.19,0.90,1.49,1.53,1.43,-0.81,0.27,0.88,0.03,1.00,0.11,0.54,0.33,0.06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625.00,1.00,0.02,0.04,1.12,1.17,-1.02,-2.19,-0.28,-3.25,-0.10,0.05,-0.04,2.06,1.37,1.10,0.08,0.05,0.07,0.05,1.11,-0.18,-0.18,0.25,-0.47,0.31,1.14,-0.87,3.12,3.08,3.28,-0.69,-0.69,-0.69,2.77,-1.44,-4.00,-0.34,0.12,0.24,0.14,0.04,0.07,-0.28,-0.47,-0.51,-0.21,0.34,0.02,0.02,0.02,0.23,-0.73,-0.73,-0.73,0.86,0.82,0.85,0.42,0.28,0.13,-0.29,-0.29,-0.18,0.00,0.83,0.00,0.00,0.24,0.34,0.78,0.59,1.33,1.60,-0.19,0.22,0.22,0.22,0.46,0.22,0.00
105438,173659.00,1.00,0.01,0.02,1.14,1.12,1.17,-1.78,-2.19,-0.28,0.34,0.05,0.40,-0.04,-2.43,0.29,0.08,0.25,0.05,0.07,-0.18,1.11,-0.18,0.25,-0.18,0.39,-1.37,-0.66,2.78,3.12,3.08,-0.69,-0.69,-0.69,-0.12,-0.83,1.31,-0.13,-0.34,0.12,0.18,0.14,0.04,0.98,-0.28,-0.47,-0.21,0.55,-0.04,0.02,0.02,0.23,-0.73,-0.73,-0.73,0.02,0.86,0.82,0.22,0.06,-0.71,-0.18,-0.29,-0.29,0.00,0.00,0.00,0.00,-0.78,0.00,0.34,0.78,0.69,1.33,0.09,0.22,0.22,0.22,0.25,0.46,-0.01
105439,173670.00,1.00,0.01,0.01,-0.30,1.14,1.12,-2.17,-1.78,-2.19,0.18,0.05,-0.04,0.40,-3.83,0.34,0.08,0.19,0.25,0.05,-0.18,

In [60]:
df_lag = df_lag.dropna()
df_lag

,row_id,f84,f82,f82_lag_1,f24,f24_lag_1,f24_lag_2,f16,f16_lag_1,f16_lag_2,f8,f64,f59,f59_lag_1,f23,f2,f15,f30,f30_lag_1,f30_lag_2,f3,f3_lag_1,f3_lag_2,f21,f27,f51,f1,f57,f35,f35_lag_1,f35_lag_2,f13,f13_lag_1,f13_lag_2,f52,f39,f53,f28,f28_lag_1,f28_lag_2,f81,f81_lag_1,f81_lag_2,f87,f87_lag_1,f87_lag_2,f14,f43,f36,f36_lag_1,f36_lag_2,f69,f42,f42_lag_1,f42_lag_2,f32,f32_lag_1,f32_lag_2,f74,f75,f77,f71,f71_lag_1,f71_lag_2,f5,f5_lag_1,f66,f66_lag_1,f58,f34,f34_lag_1,f34_lag_2,f18,f18_lag_1,f17,f63,f63_lag_1,f63_lag_2,f83,f83_lag_1,target
2,3.00,-0.73,0.00,0.00,-0.22,-0.16,-0.22,-3.18,-0.76,-0.72,-0.66,0.22,-0.85,-0.77,-0.49,0.99,0.59,0.09,0.10,0.41,0.04,-0.34,-0.23,-0.24,-0.00,0.16,-3.73,1.52,1.01,3.02,-1.66,0.50,0.67,0.30,-0.20,0.99,0.44,-0.17,-0.14,-0.17,0.04,0.16,0.03,0.36,-0.22,0.99,1.00,0.46,-0.26,-0.26,-0.63,0.18,0.67,0.66,0.71,0.45,0.49,0.44,0.76,0.45,0.15,-1.88,-2.00,-2.00,0.16,0.27,-0.08,-0.11,0.68,1.43,1.24,1.52,-2.74,2.01,-0.71,0.11,0.01,0.02,0.18,0.19,0.00
3,4.00,-0.40,0.00,0.00,-0.22,-0.22,-0.16,-3.99,-3.18,-0.76,-0.27,-0.05,2.46,-0.85,-0.20,0.04,0.43,0.09,0.09,0.10,0.31,0.04,-0.34,-0.98,-0.11,0.16,2.12,1.41,1.90,1.01,3.02,0.30,0.50,0.67,-0.39,0.10,3.45,-0.09,-0.17,-0.14,0.08,0.04,0.16,0.01,0.36,-0.22,-0.22,0.65,0.24,-0.26,-0.26,0.18,0.81,0.67,0.66,0.44,0.45,0.49,0.55,0.45,-0.37,-1.58,-1.88,-2.00,0.76,0.16,-0.19,-0.08,0.02,1.53,1.43,1.24,0.27,-2.74,0.73,1.00,0.11,0.01,0.33,0.18,-0.03
4,5.00,0.51,0.00,0.00,-0.22,-0.22,-0.22,-2.02,-3.99,-3.18,1.56,-0.04,-1.49,2.46,-0.03,-0.02,0.32,0.09,0.09,0.09,0.04,0.31,0.04,0.42,-0.19,0.16,-0.85,0.93,1.88,1.90,1.01,1.15,0.30,0.50,-0.27,0.70,1.20,0.01,-0.09,-0.17,0.05,0.08,0.04,-0.01,0.01,0.36,0.52,2.47,-0.03,0.24,-0.26,0.18,0.87,0.81,0.67,0.43,0.44,0.45,0.34,0.45,1.64,-1.63,-1.58,-1.88,0.12,0.76,-0.19,-0.19,0.90,1.49,1.53,1.43,-0.81,0.27,0.88,0.03,1.00,0.11,0.54,0.33,0.06
5,6.00,1.60,0.00,0.00,-0.22,-0.22,-0.22,-1.25,-2.02,-3.99,-1.14,0.06,1.09,-1.49,-0.20,0.92,0.54,0.08,0.09,0.09,-0.22,0.04,0.31,-0.40,-0.11,0.16,0.62,-1.48,3.06,1.88,1.90,0.84,1.15,0.30,-0.39,0.48,1.46,-0.08,0.01,-0.09,0.03,0.05,0.08,-0.21,-0.01,0.01,0.57,0.35,-0.08,-0.03,0.24,0.18,0.48,0.87,0.81,0.51,0.43,0.44,0.31,0.45,-1.00,-1.58,-1.63,-1.58,0.22,0.12,0.00,-0.19,0.52,1.78,1.49,1.53,0.77,-0.81,-0.22,0.23,0.03,1.00,0.52,0.54,-0.09
6,7.00,2.07,0.00,0.00,-0.13,-0.22,-0.22,-1.09,-1.25,-2.02,-0.20,-0.21,0.36,1.09,-0.57,1.02,0.25,0.09,0.08,0.09,-0.79,-0.22,0.04,-0.25,-0.01,0.19,-0.76,1.11,3.25,3.06,1.88,1.03,0.84,1.15,-0.34,0.33,-3.33,-0.22,-0.08,0.01,0.11,0.03,0.05,-0.13,-0.21,-0.01,0.38,-1.30,-0.16,-0.08,-0.03,0.16,0.62,0.48,0.87,0.47,0.51,0.43,0.22,0.44,1.71,-1.74,-1.58,-1.63,0.36,0.22,-0.10,0.00,0.30,1.82,1.78,1.49,-0.17,0.77,-0.42,0.12,0.23,0.03,0.61,0.52,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105437,173625.00,1.00,0.02,0.04,1.12,1.17,-1.02,-2.19,-0.28,-3.25,-0.10,0.05,-0.04,2.06,1.37,1.10,0.08,0.05,0.07,0.05,1.11,-0.18,-0.18,0.25,-0.47,0.31,1.14,-0.87,3.12,3.08,3.28,-0.69,-0.69,-0.69,2.77,-1.44,-4.00,-0.34,0.12,0.24,0.14,0.04,0.07,-0.28,-0.47,-0.51,-0.21,0.34,0.02,0.02,0.02,0.23,-0.73,-0.73,-0.73,0.86,0.82,0.85,0.42,0.28,0.13,-0.29,-0.29,-0.18,0.00,0.83,0.00,0.00,0.24,0.34,0.78,0.59,1.33,1.60,-0.19,0.22,0.22,0.22,0.46,0.22,0.00
105438,173659.00,1.00,0.01,0.02,1.14,1.12,1.17,-1.78,-2.19,-0.28,0.34,0.05,0.40,-0.04,-2.43,0.29,0.08,0.25,0.05,0.07,-0.18,1.11,-0.18,0.25,-0.18,0.39,-1.37,-0.66,2.78,3.12,3.08,-0.69,-0.69,-0.69,-0.12,-0.83,1.31,-0.13,-0.34,0.12,0.18,0.14,0.04,0.98,-0.28,-0.47,-0.21,0.55,-0.04,0.02,0.02,0.23,-0.73,-0.73,-0.73,0.02,0.86,0.82,0.22,0.06,-0.71,-0.18,-0.29,-0.29,0.00,0.00,0.00,0.00,-0.78,0.00,0.34,0.78,0.69,1.33,0.09,0.22,0.22,0.22,0.25,0.46,-0.01
105439,173670.00,1.00,0.01,0.01,-0.30,1.14,1.12,-2.17,-1.78,-2

In [69]:
features = df_lag.drop(columns=['row_id']).iloc[:, :-34]
target = df_lag['target']

# Normalize the features
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

In [70]:
# Parameters
sequence_length = 10

# Create sequences
X, y = create_sequences(features_scaled, target.values, sequence_length)

# Split into training and test sets
split_ratio = 0.8
split_index = int(len(X) * split_ratio)
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

In [71]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(sequence_length, features.shape[1])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_6 (LSTM)                   │ (None, 50)             │        19,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,451 (75.98 KB)

 Trainable params: 19,451 (75.98 KB)

 Non-trainable params: 0 (0.00 B)

In [72]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - loss: 0.0092 - val_loss: 0.0060
Epoch 2/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0040 - val_loss: 0.0058
Epoch 3/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0039 - val_loss: 0.0058
Epoch 4/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0040 - val_loss: 0.0057
Epoch 5/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0039 - val_loss: 0.0058
Epoch 6/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0039 - val_loss: 0.0058
Epoch 7/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0038 - val_loss: 0.0059
Epoch 8/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0037 - val_loss: 0.0059
Epoch 9/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0037 - val_loss: 0.0061
Epoch 10/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0036 - val_loss: 0.0062
Epoch 11/50
2636/2636 ━━━━━━━━━━━━━━━━━━━━ 15s 6ms/step - loss: 0.0035 - val_loss: 0.0064
Epoch 12/50
2636/26

In [ ]:
# Build a neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(64, activation='tanh'))
model.add(BatchNormalization()) 
model.add(Dense(1))

# optimizer = Adam(learning_rate=0.007)  
model.compile(optimizer='adam', loss='mse')
model.summary()

In [118]:
features = df_lag.drop(columns=['row_id', 'target'])
target = df_lag['target']

# Normalize the features
scaler = MinMaxScaler(feature_range=(-1, 1))
features_scaled = scaler.fit_transform(features)

In [119]:
# Split data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(features, target, test_size=0.25, random_state=42)

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))